In [ ]:
from trackml.dataset import load_event
from trackml.utils import add_position_quantities, add_momentum_quantities, decode_particle_id

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from tqdm import tqdm

path = '/nfs/fanae/user/uo290313/Desktop/TFG/dataset/trackml-particle-identification/train_1/'
event = 'event000001000'

print('Loading event {}'.format(path+event))

hits, cells, particles, truth = load_event(path+event)

mem_bytes = (hits.memory_usage(index=True).sum() 
             + cells.memory_usage(index=True).sum() 
             + particles.memory_usage(index=True).sum() 
             + truth.memory_usage(index=True).sum())
print('{} memory usage {:.2f} MB'.format(event, mem_bytes / 2**20))

In [ ]:
# '''
# Representación de los hits
# '''
# print(hits.head())

# g = sns.jointplot(x=hits.x, y=hits.y, s=1, height=6)
# g.ax_joint.cla()
# plt.sca(g.ax_joint)

# volumes = hits.volume_id.unique()
# for volume in volumes:
#     v = hits[hits.volume_id == volume]
#     plt.scatter(v.x, v.y, s=3, label='volume {}'.format(volume))

# plt.xlabel('X (mm)')
# plt.ylabel('Y (mm)')
# plt.legend()
# plt.show()
# # Iterar sobre los volúmenes únicos
# for volume in hits['volume_id'].unique():  
#     fig = plt.figure(figsize=(12, 8))
#     ax = fig.add_subplot(111, projection='3d')
#     hits_volume = hits[hits['volume_id'] == volume]  # Filtrar los hits por volumen
#     ax.scatter(hits_volume['x'], hits_volume['y'], hits_volume['z'], 
#                label=f'Volume {volume}', s=5, alpha=0.75)

# ax.legend()
# ax.set_xlabel('X')
# ax.set_ylabel('Y')
# ax.set_zlabel('Z')
# ax.set_title('Distribución de hits por volumen')
# plt.show()

In [ ]:
#============== CONSIDERO SOLO LOS HITS DE LA PARTE CENTRAL DEL DETECTOR ==============#

def distance(particle):
    ''' Distancia en mm de la partícula al origen'''
    return np.sqrt(particle.vx**2 + particle.vy**2 + particle.vz**2)

particles['r'] = distance(particles)
particles['phi'] = np.arctan2(particles.vy, particles.vx)
particles['theta'] = np.arccos(particles.vz / particles.r)

print(particles.head())

# Voy a coger solo las partículas con r < 2.6
particles_all = particles
particles = particles[particles.r < 2.6]

# Con ese radio, voy a coger solo las partículas con z entre -25 y 25 mm
particles = particles[(particles.vz > -25) & (particles.vz < 25)]

# # Histograma normalizado a 1 de la variable r
# plt.figure(figsize=(6, 6))
# plt.hist(particles_all.r, bins=100, range=(0, 100), density=False, alpha=0.5, label='All particles')
# plt.hist(particles.r, bins=100, range=(0, 100), density=False, alpha=0.5, label='Selected particles')
# plt.legend()
# plt.xlabel('r (mm)')    
# plt.ylabel('Number of particles')
# plt.grid(linestyle='--', alpha=0.6)
# plt.show()

# Del truth cojo solo las partículas que están en particles
truth = truth[truth.particle_id.isin(particles.particle_id)]

# Cojo ahora los hits_id que están en truth
hits_all = hits
hits = hits[hits.hit_id.isin(truth.hit_id)]
print(hits.head())

print("Los datos que tomo son un {:.4f}% de los datos originales".format(hits.shape[0]/hits_all.shape[0]*100))


# # Represento el dataset de particles en 3D
# fig = plt.figure(figsize=(6, 6))
# plt.suptitle('Particles')
# ax = fig.add_subplot(111, projection='3d')
# ax.plot(particles.vx, particles.vy, particles.vz, 'o', markersize=1, label = 'Partículas seleccionadas')
# #ax.plot(particles_all.vx, particles_all.vy, particles_all.vz, 'x', alpha  = .6, markersize = .6, label = 'Todas las partículas')
# ax.plot(hits.x, hits.y, hits.z, 'o', markersize=.2, alpha= .4, label = 'Hits correspondientes')
# ax.legend(loc = 'best')
# ax.set_xlabel('X (mm)')
# ax.set_ylabel('Y (mm)')
# ax.set_zlabel('Z (mm)')
# plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import importlib
import kalman_filter
importlib.reload(kalman_filter)
import top_momentum_module
importlib.reload(top_momentum_module)
from kalman_filter import apply_kalmanfilter
from top_momentum_module import top_momentum


# ======== PARÁMETROS CONFIGURABLES ========
DT = 1
COS_THRESHOLD = 0
Q_COEFF = 0.1
REDUCTION_FRACTION = 0.3

# TOP_PARTICLES = True
# if TOP_PARTICLES:
#     hits, truth, top_particles = top_momentum(path, event)
#     REDUCTION_FRACTION = 1.0
#     if (top_particles['pt'] > 3).any():  # si hay alguna con pt > x
#         Q_COEFF = 0.01

TRACKS_TO_DRAW = 30

VISUALIZAR = True
HITS_CERCANOS = True

SMOOTHING = True

OCTANTE = False

# Matrices Kalman
F = np.array([[1, DT, 0,  0,  0,  0],
              [0,  1,  0,  0,  0,  0],
              [0,  0,  1, DT,  0,  0],
              [0,  0,  0,  1,  0,  0],
              [0,  0,  0,  0,  1, DT],
              [0,  0,  0,  0,  0,  1]])

H = np.array([[1, 0, 0, 0, 0, 0],
              [0, 0, 1, 0, 0, 0],
              [0, 0, 0, 0, 1, 0]])

C = np.eye(6) * 1e-3
Q = np.eye(6) * 1e-4    #1e-2 mejor candidato
R = np.eye(3) * 1e-4    #1e-2 mejor candidato


# ======== FUNCIONES AUXILIARES ========
from kalman_filter import campo_magnetico, apply_lorentz_correction, get_initial_state, get_hits_dict, cos_angle

# ======== PREPARACÍON DE DATOS Y CONSTRUCCIÓN DE TRACKS ========
from iter_trayectorias import track_finding
hits, cells, particles, truth = load_event(path+event)
hits, particles, truth = top_momentum(hits, particles, truth)
tracks, hits_vecinos_por_track, top_particles, volume_ids, hits_dict_all_volumes, truth = track_finding(hits, truth, particles, 4, np.inf)

# ======== VISUALIZACIÓN DE TRACKS ========    
from kalman_filter import visualizar_3D_hits_y_tracks
visualizar_3D_hits_y_tracks(volume_ids, hits_dict_all_volumes, tracks, hits_vecinos_por_track)


In [ ]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from trackml.randomize import shuffle_hits
from trackml.score import score_event


# Filtrar solo los volúmenes en el DataFrame truth
truth_con_volumen = truth.merge(hits[['hit_id', 'volume_id']], on='hit_id', how='left')
truth_filtrado = truth_con_volumen[truth_con_volumen['volume_id'].isin(volume_ids)]
truth_filtrado = truth_filtrado[truth_filtrado['particle_id'] != 0]

def construir_trayectorias_truth_weight(truth_df, min_hits_por_particula=2, OCTANTE=False):
    trayectorias_truth = []
    weights_truth = []
    grouped = truth_df.groupby('particle_id')

    for pid, grupo in grouped:
        if len(grupo) >= min_hits_por_particula:
            grupo_ordenado = grupo.sort_values('tz')
            tray = grupo_ordenado[['tx', 'ty', 'tz']].values
            weights = grupo_ordenado['weight'].values
            vertex = [0,0,0]
            # Filtro angular
            v1 = tray[0] - vertex
            v2 = tray[1] - vertex if len(tray) > 1 else tray[-1] - vertex
            if cos_angle(v1, v2) < COS_THRESHOLD:
                continue

            # Filtrado por octante
            if OCTANTE:
                tray = tray[(tray[:, 0] > 0) & (tray[:, 1] > 0) & (tray[:, 2] > 0)]  # Filtro por primer octante

            trayectorias_truth.append(tray)
            weights_truth.append(weights)

    return trayectorias_truth, weights_truth


# Importamos las métricas
from metric_scores import valid_tracks_from_best_hits
min_ratio = 0.5

# Construir trayectorias y pesos del truth
trayectorias_truth, pesos_truth = construir_trayectorias_truth_weight(truth_filtrado)
print(f"Se reconstruyeron {len(trayectorias_truth)} trayectorias del truth.")
# Maximo de hits por trayectoria
max_hits = max([len(tray) for tray in trayectorias_truth])
print(f"Máximo de hits por trayectoria: {max_hits}")

import plots_trayectorias
importlib.reload(plots_trayectorias)
from plots_trayectorias import plot_tracks_and_truth_2x2
plot_tracks_and_truth_2x2(tracks, trayectorias_truth)

# plt.figure(figsize=(12, 6))
# plt.hist([len(tray) for tray in trayectorias_truth], bins=50)
# plt.title('Distribución de la cantidad de hits por trayectoria del truth')
# plt.xlabel('Cantidad de hits por trayectoria')
# plt.ylabel('Frecuencia')   
# plt.grid(linestyle='--', alpha=0.6)
# plt.show()

# Media de hits por trayectoria
media_hits = np.mean([len(tray) for tray in trayectorias_truth])
print(f"Media de hits por trayectoria: {media_hits:.2f}")

print(truth_filtrado['particle_id'].value_counts().head(10))

# Mezclar los hits del truth
shuffled_truth = shuffle_hits(truth_filtrado, 0.05)

valid_tracks, valid_track_indices, track_to_pid = valid_tracks_from_best_hits(
    hits_vecinos_por_track, truth_filtrado, min_ratio=min_ratio)

total_tracks_reconstruidos = len(hits_vecinos_por_track)
precision = valid_tracks / total_tracks_reconstruidos * 100
# recall = valid_tracks / len(trayectorias_truth) * 100

print(f"Precision: {precision:.2f}%")
# print(f"Recall: {recall:.2f}%")
# f1_score = 2 * (precision * recall) / (precision + recall)
# print(f"F1 Score: {f1_score:.2f}%")


# Score de referencia usando hits mezclados
score_with_shuffled = score_event(truth_filtrado, shuffled_truth)
if isinstance(score_with_shuffled, (float, np.float32)):
    print(f"Puntuación máxima posible: {score_with_shuffled:.4f}")
else:
    print("Puntuación máxima posible:")
    for metric, value in score_with_shuffled.items():
        print(f"{metric}: {value:.4f}")

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt

# ============================================
#  Selección ÚNICA de trayectorias a plotear
# ============================================
N = TRACKS_TO_DRAW
N = 100
valid_indices = list(valid_track_indices)
invalid_indices = list(set(range(len(tracks))) - set(valid_indices))

num_valid = min(int(0.8 * N), len(valid_indices))
num_invalid = max(min(N - num_valid, len(invalid_indices)), 0)

if len(valid_indices) < int(0.8 * N):
    print(f"Advertencia: solo {len(valid_indices)} trayectorias buenas disponibles, se completará con malas.")

print(f"Trayectorias válidas: {len(valid_indices)}")
print(f"Trayectorias inválidas: {len(invalid_indices)}")
print(f"Trayectorias válidas a graficar: {num_valid}")
print(f"Trayectorias inválidas a graficar: {num_invalid}")

sampled_valids = random.sample(valid_indices, num_valid)
sampled_invalids = random.sample(invalid_indices, num_invalid)

indices_to_plot = sampled_valids + sampled_invalids
random.shuffle(indices_to_plot)
indices_to_plot = [i for i in indices_to_plot if i < len(trayectorias_truth)]

print(f'Indices únicos a graficar (filtrados): {indices_to_plot}')

# ============================================
# === GRAFICAR EN 3D
# ============================================
GRAFICAR_TRUTH = True
if GRAFICAR_TRUTH:
    %matplotlib qt
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')

    for volume_id in volume_ids:
        for layer in hits_dict_all_volumes[volume_id].keys():  
            layer_hits = hits_dict_all_volumes[volume_id][layer]
            ax.scatter(layer_hits['x'], layer_hits['y'], layer_hits['z'], s=3, alpha=0.3)

    label_kalman_done = False
    label_buenas_done = False
    label_truth_done = False

    for idx in indices_to_plot:
        track = tracks[idx]

        # Verificar si hay un truth asociado
        if idx in track_to_pid:
            pid = track_to_pid[idx]
            tray_truth_df = truth_filtrado[truth_filtrado['particle_id'] == pid].sort_values('tz')
            tray_truth = tray_truth_df[['tx', 'ty', 'tz']].values
        else:
            tray_truth = np.zeros((0, 3))

        is_valid = idx in valid_track_indices
        track_color = 'tab:green' if is_valid else 'tab:red'

        if is_valid and not label_buenas_done:
            label_kalman = "Buenas"
            label_buenas_done = True
        elif not is_valid and not label_kalman_done:
            label_kalman = "Kalman"
            label_kalman_done = True
        else:
            label_kalman = ""

        if not label_truth_done:
            label_truth = "Truth"
            label_truth_done = True
        else:
            label_truth = ""

        ax.plot(track[:, 0], track[:, 1], track[:, 2], color=track_color, alpha=0.7, label=label_kalman)
        if len(tray_truth) > 0:
            ax.plot(tray_truth[:, 0], tray_truth[:, 1], tray_truth[:, 2], color='blue', alpha=0.7, linestyle='-', label=label_truth)


    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('Trayectorias del truth y Kalman')
    ax.legend()
    plt.show()

# ============================================
# === GRAFICAR EN 2D
# ============================================
def plot_xy_projection(tracks, truth_filtrado, valid_track_indices, track_to_pid, indices_to_plot):
    fig, ax = plt.subplots(figsize=(8, 8))

    label_kalman_done = False
    label_buenas_done = False
    label_truth_done = False

    for idx in indices_to_plot:
        track = tracks[idx]

        if idx in track_to_pid:
            pid = track_to_pid[idx]
            tray_truth_df = truth_filtrado[truth_filtrado['particle_id'] == pid].sort_values('tz')
            tray_truth = tray_truth_df[['tx', 'ty']].values
        else:
            tray_truth = np.zeros((0, 2))

        is_valid = idx in valid_track_indices
        track_color = 'tab:green' if is_valid else 'tab:red'

        if is_valid and not label_buenas_done:
            label_kalman = "Buenas"
            label_buenas_done = True
        elif not is_valid and not label_kalman_done:
            label_kalman = "Kalman"
            label_kalman_done = True
        else:
            label_kalman = ""

        if not label_truth_done:
            label_truth = "Truth"
            label_truth_done = True
        else:
            label_truth = ""

        ax.plot(track[:, 0], track[:, 1], color=track_color, alpha=0.7, label=label_kalman)
        ax.scatter(track[:, 0], track[:, 1], s=10, color=track_color, alpha=0.3)

        if len(tray_truth) > 0:
            ax.plot(tray_truth[:, 0], tray_truth[:, 1], color='blue', alpha=0.7, linestyle='-', label=label_truth)
            ax.scatter(tray_truth[:, 0], tray_truth[:, 1], s=10, color='blue', alpha=0.3)

    ax.legend()
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_title('Proyección XY de las trayectorias del truth y Kalman')
    ax.axis('equal')
    ax.text(0.98, 0.15, f'min_ratio: {min_ratio}', transform=ax.transAxes,
            fontsize=12, verticalalignment='top', horizontalalignment = 'right', bbox=dict(boxstyle='round', facecolor='white', alpha=0.5))  
    ax.text(0.98, 0.10, f'Precision: {precision:.4f}%', transform=ax.transAxes,
            fontsize=12, verticalalignment='top', horizontalalignment = 'right', bbox=dict(boxstyle='round', facecolor='white', alpha=0.5))
    ax.text(0.98, 0.05, r'$cos_{lim}(v_1, v_2)$ = ' f'{COS_THRESHOLD}', transform=ax.transAxes,
            fontsize=12, verticalalignment='top', horizontalalignment = 'right', bbox=dict(boxstyle='round', facecolor='white', alpha=0.5))
    plt.show()


# Finalmente, llamar a la función para graficar en 2D
if GRAFICAR_TRUTH:
    plot_xy_projection(tracks, truth_filtrado, valid_track_indices, track_to_pid, indices_to_plot=indices_to_plot)


In [ ]:
# Dibujar todos los tracks en XY
def plot_all_tracks(tracks, ALL_TRACKS=False):   
    if ALL_TRACKS:
        fig, ax = plt.subplots(figsize=(8, 8))
        for track in tracks:
            ax.plot(track[:, 0], track[:, 1], alpha=0.5)
        ax.set_title('Todos los tracks en XY')
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.axis('equal')  
        plt.show()
plot_all_tracks(tracks)

In [ ]:
GRAFICAR_DISTANCIAS = False
if GRAFICAR_DISTANCIAS:
    all_dists = []

    fig = plt.figure(figsize=(12, 8))
    for k_track in tracks:
        for t_track in trayectorias_truth:
            d = cdist(k_track, t_track)
            all_dists.extend(np.min(d, axis=1))  # Solo las mínimas distancias

    plt.hist(all_dists, bins=100)
    plt.xlabel("Distancia mínima punto a punto")
    plt.title("Distribución de distancias entre trayectorias predichas y reales")
    plt.grid()
    plt.show()

# SECCIÓN ITERATIVA PARA DISTINTOS ESCENARIOS DE INTERVALOS DE pt_min Y pt_max

In [46]:
from tqdm import tqdm
import numpy as np
import top_momentum_module
import iter_trayectorias
importlib.reload(top_momentum_module)
importlib.reload(iter_trayectorias)
from top_momentum_module import top_momentum
from iter_trayectorias import track_finding
from trackml.dataset import load_event


# Cargar datos una sola vez
hits, cells, particles, truth = load_event(path + event)

# Quitamos el ruido
particles = particles[particles.particle_id != 0]

# Añadir pT a partículas
particles['pt'] = np.sqrt(particles['px']**2 + particles['py']**2)

# Inicializar variables de trabajo
hits_remaining = hits.copy()
truth_remaining = truth.copy()
all_results = []

pt_edges = [0, 0.05, 0.1, 0.25, 0.35, 0.5, 0.75, 1.0, 2.0, 3.0, np.inf]
#pt_edges = [3.0, np.inf]


for i in tqdm(reversed(range(len(pt_edges) - 1)), desc="Procesando pt bins"):
    pt_min = pt_edges[i]
    pt_max = pt_edges[i+1]

    print(f"\n=== Procesando pT ∈ [{pt_min}, {pt_max}) GeV/c ===")

    # Filtrar datos con top_momentum
    hits_filtered, top_particles, truth_filtered  = top_momentum(
        hits_remaining.copy(), particles.copy(), truth_remaining.copy(), pt_min=pt_min, pt_max=pt_max
    )

    if hits_filtered.empty:
        print("→ No hay hits válidos en este rango.")
        continue

    # Ejecutar el seguimiento
    tracks, hits_vecinos_por_track, top_particles, volume_ids, hits_dict_all_volumes, truth = track_finding(
        hits_filtered, truth_filtered, top_particles, pt_min, pt_max
    )

    all_results.append({
        "pt_range": (pt_min, pt_max),
        "tracks": tracks,
        "hits_vecinos": hits_vecinos_por_track,
        "particles": top_particles,
        "truth": truth_filtered
    })
    # Construir trayectorias del truth y pesos
    trayectorias_truth, pesos_truth = construir_trayectorias_truth_weight(truth_filtered)
    all_results[-1]["trayectorias_truth"] = trayectorias_truth
    all_results[-1]["pesos_truth"] = pesos_truth

    # Calcular precisión
    valid_tracks, valid_track_indices, track_to_pid = valid_tracks_from_best_hits(
        hits_vecinos_por_track, truth_filtered, min_ratio=min_ratio
    )

    total_tracks = len(hits_vecinos_por_track)
    precision = valid_tracks / total_tracks * 100 if total_tracks > 0 else 0.0

    all_results[-1]["precision"] = precision
    all_results[-1]["valid_tracks"] = valid_tracks
    all_results[-1]["total_tracks"] = total_tracks

    print(f"→ Precisión: {precision:.2f}%")

    # Actualizar hits y truth para la siguiente iteración
    hits_usados = hits_filtered.hit_id.unique()
    truth_usados = truth_filtered.hit_id.unique()

    # Quitar hits y truths ya utilizados
    hits_remaining = hits_remaining[~hits_remaining.hit_id.isin(hits_usados)]
    truth_remaining = truth_remaining[~truth_remaining.hit_id.isin(truth_usados)]

print("\nResumen de precisión por bin de pT:")
for res in all_results:
    pt_min, pt_max = res["pt_range"]
    print(f"pT ∈ [{pt_min}, {pt_max}): {res['precision']:.2f}% "
        f"({res['valid_tracks']} / {res['total_tracks']}) tracks válidos")


Procesando pt bins: 0it [00:00, ?it/s]


=== Procesando pT ∈ [3.0, inf) GeV/c ===
Partículas seleccionadas: 61
pT max = 45.3548 GeV/c, pT min = 3.0348 GeV/c
Los datos que tomo son un 0.5193% de los datos originales
628 hits seleccionados
→ particle_ids únicos: 61
→ truth con esos particle_ids: 628
→ hits con esos hit_id: 628


Procesando pt bins: 1it [00:00,  1.10it/s]

Rango pT: 3.00 - inf GeV/c
Total tracks: 62, Positivos: 26, Negativos: 36
→ Precisión: 83.87%

=== Procesando pT ∈ [2.0, 3.0) GeV/c ===
Partículas seleccionadas: 75
pT max = 2.9759 GeV/c, pT min = 2.0154 GeV/c
Los datos que tomo son un 0.6749% de los datos originales
812 hits seleccionados
→ particle_ids únicos: 75
→ truth con esos particle_ids: 812
→ hits con esos hit_id: 812


Procesando pt bins: 2it [00:01,  1.02it/s]

...

→ Precisión: 69.33%

=== Procesando pT ∈ [0.25, 0.35) GeV/c ===
Partículas seleccionadas: 512
pT max = 0.3499 GeV/c, pT min = 0.2502 GeV/c
Los datos que tomo son un 5.7956% de los datos originales
5593 hits seleccionados
→ particle_ids únicos: 512
→ truth con esos particle_ids: 5593
→ hits con esos hit_id: 5593
Rango pT: 0.25 - 0.35 GeV/c
Total tracks: 493, Positivos: 281, Negativos: 212


Procesando pt bins: 7it [00:39,  7.18s/it]

→ Precisión: 39.96%

=== Procesando pT ∈ [0.1, 0.25) GeV/c ===
Partículas seleccionadas: 776
pT max = 0.2500 GeV/c, pT min = 0.1006 GeV/c
Los datos que tomo son un 6.1323% de los datos originales
5575 hits seleccionados
→ particle_ids únicos: 776
→ truth con esos particle_ids: 5575
→ hits con esos hit_id: 5575
Rango pT: 0.10 - 0.25 GeV/c
Total tracks: 544, Positivos: 243, Negativos: 301


Procesando pt bins: 10it [00:47,  4.74s/it]

→ Precisión: 17.65%

=== Procesando pT ∈ [0.05, 0.1) GeV/c ===
Partículas seleccionadas: 0
Los datos que tomo son un 0.0000% de los datos originales
0 hits seleccionados
→ particle_ids únicos: 0
→ truth con esos particle_ids: 0
→ hits con esos hit_id: 0
→ No hay hits válidos en este rango.

=== Procesando pT ∈ [0, 0.05) GeV/c ===
Partículas seleccionadas: 0
Los datos que tomo son un 0.0000% de los datos originales
0 hits seleccionados
→ particle_ids únicos: 0
→ truth con esos particle_ids: 0
→ hits con esos hit_id: 0
→ No hay hits válidos en este rango.

Resumen de precisión por bin de pT:
pT ∈ [3.0, inf): 83.87% (52 / 62) tracks válidos
pT ∈ [2.0, 3.0): 93.15% (68 / 73) tracks válidos
pT ∈ [1.0, 2.0): 82.60% (356 / 431) tracks válidos
pT ∈ [0.75, 1.0): 83.42% (307 / 368) tracks válidos
pT ∈ [0.5, 0.75): 67.27% (481 / 715) tracks válidos
pT ∈ [0.35, 0.5): 69.33% (425 / 613) tracks válidos
pT ∈ [0.25, 0.35): 39.96% (197 / 493) tracks válidos
pT ∈ [0.1, 0.25): 17.65% (96 / 544) tracks vá

## Calculamos la precisión global, dibujamos trayectorias kalman y truth asociadas

In [47]:
# Juntar todos los datos
all_tracks = []
all_truth = []
all_valid_track_indices = []
all_track_to_pid = {}
offset = 0

for res in all_results:
    all_tracks.extend(res["tracks"])
    all_truth.append(res["truth"])

    # Obtener valids para cada bin
    valid_tracks, valid_indices, track_to_pid = valid_tracks_from_best_hits(
        res["hits_vecinos"], res["truth"], min_ratio=min_ratio
    )

    # Ajustar índices (porque los índices de hits_vecinos son locales por bin)
    valid_indices_offset = [idx + offset for idx in valid_indices]
    all_valid_track_indices.extend(valid_indices_offset)

    # Ajustar track_to_pid con offset
    for idx, pid in track_to_pid.items():
        all_track_to_pid[idx + offset] = pid

    offset += len(res["hits_vecinos"])

# Concatenar truth total
truth_total = pd.concat(all_truth, ignore_index=True)

total_tracks = len(all_tracks)
valid_tracks_total = len(all_valid_track_indices)
precision_global = valid_tracks_total / total_tracks * 100

print(f"\n🎯 Precisión global: {precision_global:.2f}% ({valid_tracks_total} / {total_tracks})")



🎯 Precisión global: 60.08% (1982 / 3299)


In [48]:
# ==========================
# Seleccionar trayectorias
# ==========================
N = 100
valid_indices = all_valid_track_indices
invalid_indices = list(set(range(len(all_tracks))) - set(valid_indices))

num_valid = min(int(0.8 * N), len(valid_indices))
num_invalid = max(min(N - num_valid, len(invalid_indices)), 0)

sampled_valids = random.sample(valid_indices, num_valid) if num_valid > 0 else []
sampled_invalids = random.sample(invalid_indices, num_invalid) if num_invalid > 0 else []

indices_to_plot = sampled_valids + sampled_invalids
random.shuffle(indices_to_plot)


%matplotlib qt
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

label_kalman_done = False
label_buenas_done = False
label_truth_done = False

for idx in indices_to_plot:
    track = all_tracks[idx]

    # Ver si hay trayectoria truth correspondiente
    if idx in all_track_to_pid:
        pid = all_track_to_pid[idx]
        tray_truth_df = truth_total[truth_total['particle_id'] == pid].sort_values('tz')
        tray_truth = tray_truth_df[['tx', 'ty', 'tz']].values
    else:
        tray_truth = np.zeros((0, 3))

    is_valid = idx in all_valid_track_indices
    track_color = 'tab:green' if is_valid else 'tab:red'

    if is_valid and not label_buenas_done:
        label_kalman = "Buenas"
        label_buenas_done = True
    elif not is_valid and not label_kalman_done:
        label_kalman = "Kalman"
        label_kalman_done = True
    else:
        label_kalman = ""

    if len(tray_truth) > 0 and not label_truth_done:
        label_truth = "Truth"
        label_truth_done = True
    else:
        label_truth = ""

    ax.plot(track[:, 0], track[:, 1], track[:, 2], color=track_color, alpha=0.7, label=label_kalman)
    if len(tray_truth) > 0:
        ax.plot(tray_truth[:, 0], tray_truth[:, 1], tray_truth[:, 2], color='blue', alpha=0.7, linestyle='-', label=label_truth)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('Trayectorias del truth y Kalman (global)')
ax.legend()
plt.show()

def plot_xy_projection(tracks, truth_df, valid_track_indices, track_to_pid, indices_to_plot, precision_global):
    fig, ax = plt.subplots(figsize=(8, 8))
    label_kalman_done = False
    label_buenas_done = False
    label_truth_done = False

    for idx in indices_to_plot:
        track = tracks[idx]

        if idx in track_to_pid:
            pid = track_to_pid[idx]
            tray_truth_df = truth_df[truth_df['particle_id'] == pid].sort_values('tz')
            tray_truth = tray_truth_df[['tx', 'ty']].values
        else:
            tray_truth = np.zeros((0, 2))

        is_valid = idx in valid_track_indices
        track_color = 'tab:green' if is_valid else 'tab:red'

        if is_valid and not label_buenas_done:
            label_kalman = "Buenas"
            label_buenas_done = True
        elif not is_valid and not label_kalman_done:
            label_kalman = "Kalman"
            label_kalman_done = True
        else:
            label_kalman = ""

        if len(tray_truth) > 0 and not label_truth_done:
            label_truth = "Truth"
            label_truth_done = True
        else:
            label_truth = ""

        ax.plot(track[:, 0], track[:, 1], color=track_color, alpha=0.7, label=label_kalman)
        ax.scatter(track[:, 0], track[:, 1], s=10, color=track_color, alpha=0.3)

        if len(tray_truth) > 0:
            ax.plot(tray_truth[:, 0], tray_truth[:, 1], color='blue', alpha=0.7, linestyle='-', label=label_truth)
            ax.scatter(tray_truth[:, 0], tray_truth[:, 1], s=10, color='blue', alpha=0.3)

    ax.legend()
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_title('Proyección XY de trayectorias globales')
    ax.axis('equal')
    ax.text(0.98, 0.15, f'min_ratio: {min_ratio}', transform=ax.transAxes,
            fontsize=12, verticalalignment='top', horizontalalignment='right', bbox=dict(boxstyle='round', facecolor='white', alpha=0.5))
    ax.text(0.98, 0.10, f'Precision: {precision_global:.2f}%', transform=ax.transAxes,
            fontsize=12, verticalalignment='top', horizontalalignment='right', bbox=dict(boxstyle='round', facecolor='white', alpha=0.5))
    ax.text(0.98, 0.05, r'$cos_{lim}(v_1, v_2)$ = ' f'{COS_THRESHOLD}', transform=ax.transAxes,
            fontsize=12, verticalalignment='top', horizontalalignment='right', bbox=dict(boxstyle='round', facecolor='white', alpha=0.5))
    plt.show()

# Ejecutar el gráfico
plot_xy_projection(all_tracks, truth_total, all_valid_track_indices, all_track_to_pid, indices_to_plot, precision_global)
